# 2131. Longest Palindrome by Concatenating Two Letter Words

# Medium

You are given an array of strings words. Each element of words consists of two lowercase English letters.

Create the longest possible palindrome by selecting some elements from words and concatenating them in any order. Each element can be selected at most once.

Return the length of the longest palindrome that you can create. If it is impossible to create any palindrome, return 0.

> A palindrome is a string that reads the same forward and backward.

# Example 1:

```
Input: words = ["lc","cl","gg"]
Output: 6
Explanation: One longest palindrome is "lc" + "gg" + "cl" = "lcggcl", of length 6.
Note that "clgglc" is another longest palindrome that can be created.
```

# Example 2:

```
Input: words = ["ab","ty","yt","lc","cl","ab"]
Output: 8
Explanation: One longest palindrome is "ty" + "lc" + "cl" + "yt" = "tylcclyt", of length 8.
Note that "lcyttycl" is another longest palindrome that can be created.
```

# Example 3:

```
Input: words = ["cc","ll","xx"]
Output: 2
Explanation: One longest palindrome is "cc", of length 2.
Note that "ll" is another longest palindrome that can be created, and so is "xx".

```

# Constraints:

- 1 <= words.length <= 105
- words[i].length == 2
- words[i] consists of lowercase English letters.


## Problem Analysis

A palindrome reads the same forwards and backward. When concatenating two-letter words, we need to consider how they contribute to forming a palindrome.

There are two main types of two-letter words:

1.  **Words that are palindromes themselves (e.g., "gg", "cc", "aa")**: These words have the form `xx`.
2.  **Words that are not palindromes (e.g., "lc", "cl", "ab", "ba")**: These words have the form `xy` where `x != y`. For `xy` to contribute to a larger palindrome, it _must_ be paired with its reverse, `yx`. For example, `lc` needs `cl`.

A long palindrome is typically formed by:

- A central part (which can be a single word that is a palindrome itself, like "gg").
- Pairs of words that are reverses of each other, forming a structure like `word1 + word2 + ... + reverse(word2) + reverse(word1)`. For example, "ab" + "cd" + "dc" + "ba".

Crucially, if we use a word `xy` and its reverse `yx`, they contribute 4 to the total length. If we use a word `xx` and another `xx`, they contribute 4 to the total length. If we use a single `xx` word as the central part, it contributes 2 to the total length.

Let's use a frequency map (hash map or dictionary) to count the occurrences of each word. This will be central to all approaches.

## Approach 1: Using a Hash Map (Frequency Counter)

This is the most intuitive and efficient approach. We'll count the frequency of each word and then intelligently pair them up.

**Algorithm:**

1.  **Count Frequencies:** Create a hash map (dictionary in Python) to store the frequency of each word in the input `words` array.
2.  **Initialize `longest_palindrome_length`:** Set this to 0.
3.  **Initialize `has_odd_palindrome_center`:** Set this to `False`. This flag tracks if we've used a single palindrome word (like "gg") as the center of our overall palindrome. A palindrome can have at most one such central word.
4.  **Iterate through Frequencies:** Iterate through the words and their counts in the frequency map.

    - **Case A: Word is a palindrome (e.g., "gg", "cc")**:
      - Let `word` be `xx`.
      - If `count` of `xx` is even (e.g., "gg" appears 2 or 4 times): Add `count * 2` to `longest_palindrome_length`. All `xx` words can be paired up (e.g., "gg" + "gg").
      - If `count` of `xx` is odd (e.g., "gg" appears 1, 3, or 5 times): Add `(count - 1) * 2` to `longest_palindrome_length`. This accounts for all but one `xx` word, which can be paired. The remaining single `xx` word can potentially be the center of the total palindrome. Set `has_odd_palindrome_center = True`.
    - **Case B: Word is NOT a palindrome (e.g., "lc", "ab")**:
      - Let `word` be `xy`. Its reverse is `yx`.
      - Check if `yx` exists in the frequency map.
      - If `yx` exists:
        - Determine the number of pairs we can form: `num_pairs = min(freq[word], freq[reverse_word])`.
        - Add `num_pairs * 4` to `longest_palindrome_length` (each pair `xy` + `yx` contributes 4 to the length).
        - Mark both `word` and `reverse_word` as "used" or set their frequencies to 0 in the map (or simply ensure we don't process `yx` when we iterate to it later). A simple way to avoid double-counting is to remove `reverse_word` from the map or set its frequency to 0 after processing `word`.

5.  **Final Check for Central Palindrome:** If `has_odd_palindrome_center` is `True`, add 2 to `longest_palindrome_length`. This accounts for the single middle word like "gg" that we reserved.

**Code:**

```python
from collections import defaultdict

class Solution:
    def longestPalindrome(self, words: list[str]) -> int:
        freq = defaultdict(int)
        for word in words:
            freq[word] += 1

        longest_palindrome_length = 0
        has_odd_palindrome_center = False

        # Iterate through a copy of keys or use a while loop for safe deletion
        # Iterating directly over `freq.keys()` and modifying `freq` can cause issues.
        # It's safer to iterate over items and manage counts or use a different structure.

        # A common and robust way: process pairs first, then remaining single palindromes

        # Process words that are not palindromes (xy and yx pairs)
        for word, count in list(freq.items()): # Use list(freq.items()) to iterate over a copy
            if word[0] != word[1]: # Not a palindrome word like "gg"
                reverse_word = word[1] + word[0]
                if reverse_word in freq:
                    # Number of pairs we can form
                    num_pairs = min(count, freq[reverse_word])
                    longest_palindrome_length += num_pairs * 4 # Each pair contributes 4

                    # Decrement counts so they are not used again
                    freq[word] -= num_pairs
                    freq[reverse_word] -= num_pairs

        # Process words that are palindromes (xx)
        for word, count in list(freq.items()): # Iterate again, using current counts
            if word[0] == word[1]: # A palindrome word like "gg"
                # All even counts can be paired up (e.g., "gg" + "gg")
                longest_palindrome_length += (count // 2) * 4

                # If there's an odd count, one word remains. This can be the center.
                if count % 2 == 1:
                    has_odd_palindrome_center = True

        # If we have a single palindrome word available, it can be the center
        if has_odd_palindrome_center:
            longest_palindrome_length += 2

        return longest_palindrome_length

```

**Time Complexity:**

- Counting Frequencies: $O(N \times L)$, where $N$ is `len(words)` and $L$ is the length of each word (which is 2 here, so effectively $O(N)$).
- Iterating through frequencies: In the worst case, all words are unique. The number of unique words is at most $26^2 = 676$. So this loop runs for a constant number of iterations (at most 676).
- Overall Time: $O(N)$.

**Space Complexity:**

- Frequency map: $O(U)$, where $U$ is the number of unique words. $U \le 26^2 = 676$, so effectively $O(1)$ constant space relative to the input size.

## Approach 2: Simplified Hash Map Iteration (Avoiding explicit `list(freq.items())`)

The previous approach used `list(freq.items())` to avoid modifying `freq` while iterating. A slightly simpler way is to iterate through original `words` and remove items as we process them.

**Algorithm:**

1.  **Count Frequencies:** Use `collections.Counter` for convenience.
2.  **Initialize `length = 0` and `center_added = False`**.
3.  **Iterate and Process:**
    - Iterate through the `words` (or `freq.keys()`).
    - For each `word` and its `count` (from `freq`):
      - **If `word[0] == word[1]` (e.g., "gg")**:
        - Add `(count // 2) * 4` to `length`.
        - If `count % 2 == 1`, set `center_added = True`.
      - **If `word[0] != word[1]` (e.g., "lc")**:
        - Calculate `reverse_word = word[1] + word[0]`.
        - If `reverse_word` exists in `freq`:
          - `pairs = min(count, freq[reverse_word])`.
          - Add `pairs * 4` to `length`.
          - Decrement `freq[word]` and `freq[reverse_word]` by `pairs`. (This is crucial to avoid double counting and correctly handle remaining words).
4.  **Final Check:** If `center_added` is `True`, add 2 to `length`.

**Code:**

```python
from collections import Counter

class Solution:
    def longestPalindrome(self, words: list[str]) -> int:
        freq = Counter(words)

        longest_palindrome_length = 0
        has_odd_palindrome_center = False

        for word in freq: # Iterate through keys (words)
            if freq[word] == 0: # Already used up
                continue

            if word[0] == word[1]: # Palindrome word like "gg"
                longest_palindrome_length += (freq[word] // 2) * 4 # Add pairs (gg + gg)
                if freq[word] % 2 == 1:
                    has_odd_palindrome_center = True
                freq[word] = 0 # Mark as used
            else: # Non-palindrome word like "lc"
                reverse_word = word[1] + word[0]
                if reverse_word in freq and freq[reverse_word] > 0:
                    pairs = min(freq[word], freq[reverse_word])
                    longest_palindrome_length += pairs * 4
                    freq[word] -= pairs # Consume words
                    freq[reverse_word] -= pairs # Consume reverse words

        if has_odd_palindrome_center:
            longest_palindrome_length += 2

        return longest_palindrome_length

```

**Time and Space Complexity:** Same as Approach 1. This approach is slightly cleaner in how it manages the frequency counts.

## Approach 3: Categorizing and Counting

This approach explicitly categorizes words into two groups: those that are palindromes (`xx`) and those that are not (`xy`). This can sometimes make the logic clearer.

**Algorithm:**

1.  **Categorize and Count:**
    - Create `palindrome_words_freq = defaultdict(int)` for `xx` words.
    - Create `non_palindrome_words_freq = defaultdict(int)` for `xy` words.
    - Iterate through `words`:
      - If `word[0] == word[1]`, add to `palindrome_words_freq`.
      - Else, add to `non_palindrome_words_freq`.
2.  **Calculate Length from Non-Palindromes:**
    - Initialize `length = 0`.
    - Iterate through `non_palindrome_words_freq`. For each `word = xy` and its `count`:
      - Calculate `reverse_word = yx`.
      - If `reverse_word` is in `non_palindrome_words_freq` and `non_palindrome_words_freq[reverse_word] > 0`:
        - `num_pairs = min(count, non_palindrome_words_freq[reverse_word])`.
        - `length += num_pairs * 4`.
        - Decrement counts: `non_palindrome_words_freq[word] -= num_pairs`, `non_palindrome_words_freq[reverse_word] -= num_pairs`.
3.  **Calculate Length from Palindromes and Center:**
    - Initialize `has_center = False`.
    - Iterate through `palindrome_words_freq`. For each `word = xx` and its `count`:
      - `length += (count // 2) * 4`.
      - If `count % 2 == 1`, set `has_center = True`.
4.  **Final Add:** If `has_center` is `True`, add 2 to `length`.
5.  **Return `length`**.

**Code:**

```python
from collections import defaultdict

class Solution:
    def longestPalindrome(self, words: list[str]) -> int:
        palindrome_words_freq = defaultdict(int)
        non_palindrome_words_freq = defaultdict(int)

        for word in words:
            if word[0] == word[1]:
                palindrome_words_freq[word] += 1
            else:
                non_palindrome_words_freq[word] += 1

        longest_palindrome_length = 0

        # Process non-palindrome words (xy and yx pairs)
        # Iterate over a copy of items to safely modify the dictionary
        for word, count in list(non_palindrome_words_freq.items()):
            if count == 0: # Already used
                continue

            reverse_word = word[1] + word[0]

            if reverse_word in non_palindrome_words_freq and non_palindrome_words_freq[reverse_word] > 0:
                num_pairs = min(count, non_palindrome_words_freq[reverse_word])
                longest_palindrome_length += num_pairs * 4

                # Consume the words
                non_palindrome_words_freq[word] -= num_pairs
                non_palindrome_words_freq[reverse_word] -= num_pairs

        # Process palindrome words (xx) and determine if a center can be used
        has_odd_palindrome_center = False
        for word, count in palindrome_words_freq.items():
            longest_palindrome_length += (count // 2) * 4 # Add pairs of xx words (e.g., gg + gg)
            if count % 2 == 1:
                has_odd_palindrome_center = True

        if has_odd_palindrome_center:
            longest_palindrome_length += 2 # Add the single middle xx word

        return longest_palindrome_length

```

**Time and Space Complexity:** Same as Approach 1 and 2. This approach might be slightly less efficient due to two separate frequency maps, but the constant factors are small as the number of unique words is limited. The logic is very clear for understanding the two distinct types of word contributions.

## Key Considerations for all Approaches:

- **Pairs of `xy` and `yx`**: Each such pair contributes 4 to the total length. We can form `min(count(xy), count(yx))` such pairs.
- **Pairs of `xx` and `xx`**: Each such pair contributes 4 to the total length. If we have `k` occurrences of `xx`, we can form `k // 2` pairs, contributing `(k // 2) * 4` to the length.
- **Single `xx` word as center**: If after forming all possible `xx` pairs, there's one `xx` word left over (`k % 2 == 1`), this single `xx` word can be placed in the very center of the palindrome. This adds 2 to the total length. Only one such word can be used as the center.

All presented approaches correctly handle these considerations and are optimal in terms of time and space complexity given the constraints. The `Counter` based solution (Approach 2) is often the most concise and Pythonic.


In [ ]:
from collections import Counter

class Solution:
    def longestPalindrome(self, words: list[str]) -> int:
        count = Counter(words)
        length = 0
        center_used = False

        for word in count:
            rev_word = word[::-1]
            if word == rev_word:  # Palindromic words like "aa", "bb"
                pairs = count[word] // 2
                length += pairs * 4
                if count[word] % 2 == 1:
                    center_used = True
            elif rev_word in count:  # Matching pairs like "ab" and "ba"
                pairs = min(count[word], count[rev_word])
                length += pairs * 4
                count[word] = 0  # Mark as used
                count[rev_word] = 0

        if center_used:
            length += 2  # Add one palindromic word in the center

        return length

# **Test Cases**
def run_tests():
    solution = Solution()
    assert solution.longestPalindrome(["lc", "cl", "gg"]) == 6
    assert solution.longestPalindrome(["ab", "ty", "yt", "lc", "cl", "ab"]) == 8
    assert solution.longestPalindrome(["cc", "ll", "xx"]) == 2
    assert solution.longestPalindrome(["aa", "bb", "cc", "aa"]) == 6
    assert solution.longestPalindrome(["ab", "ba", "cd", "dc", "ee"]) == 10
    assert solution.longestPalindrome(["ab", "ba", "ab"]) == 4
    assert solution.longestPalindrome(["zz"]) == 2  # Single palindromic word

    print("All test cases passed!")

# Run the tests
run_tests()